# importing libraries
# 

In [7]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader   #for batch seperation
from torchvision.transforms import ToTensor  ####convert tensors to vectors
from torch.utils import data


# importing data

In [8]:
train_data  = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
train_data.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
train_data.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [10]:
train_data.select_dtypes( exclude= 'object')

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,366,0,112,0,0,0,0,4,2010,142125


In [11]:
f = train_data.select_dtypes( exclude= 'object').columns

In [12]:
train_data[f].corr()[['SalePrice']].sort_values(['SalePrice'],ascending = False)

,SalePrice
SalePrice,1.000000
OverallQual,0.790982
GrLivArea,0.708624
GarageCars,0.640409
GarageArea,0.623431
TotalBsmtSF,0.613581
1stFlrSF,0.605852
FullBath,0.560664
TotRmsAbvGrd,0.533723
YearBuilt,0.522897


In [13]:
test_data.drop_duplicates(inplace=True) 
train_data.drop_duplicates(inplace=True) 

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# removing unnecessary columns

In [15]:
train_data =train_data.drop(['Id','Street', 'Utilities','LandSlope','Condition2','RoofMatl','CentralAir','LowQualFinSF','3SsnPorch','ScreenPorch','PoolArea','MiscVal',"MiscFeature",'Fence','PoolQC','Alley','MasVnrType','FireplaceQu'], axis=1)
test_data =test_data.drop(['Id','Street', 'Utilities','LandSlope','Condition2','RoofMatl','CentralAir','LowQualFinSF','3SsnPorch','ScreenPorch','PoolArea','MiscVal',"MiscFeature",'Fence','PoolQC','Alley','MasVnrType','FireplaceQu'], axis=1)

# Merging correlated columns

In [16]:
for df in (train_data, test_data):
    df["n_bathrooms"] = df["BsmtFullBath"] + (df["BsmtHalfBath"]*0.5) + df["FullBath"] + (df["HalfBath"]*0.5)
    df["area_with_basement"]  = df["GrLivArea"] + df["TotalBsmtSF"]
    

In [17]:
train_data =train_data.drop(["BsmtFullBath","BsmtHalfBath","FullBath","HalfBath",'GrLivArea','TotalBsmtSF'], axis=1)
test_data =test_data.drop(["BsmtFullBath","BsmtHalfBath","FullBath","HalfBath",'GrLivArea','TotalBsmtSF'], axis=1)

In [18]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 59 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MSSubClass          1460 non-null   int64  
 1   MSZoning            1460 non-null   object 
 2   LotFrontage         1201 non-null   float64
 3   LotArea             1460 non-null   int64  
 4   LotShape            1460 non-null   object 
 5   LandContour         1460 non-null   object 
 6   LotConfig           1460 non-null   object 
 7   Neighborhood        1460 non-null   object 
 8   Condition1          1460 non-null   object 
 9   BldgType            1460 non-null   object 
 10  HouseStyle          1460 non-null   object 
 11  OverallQual         1460 non-null   int64  
 12  OverallCond         1460 non-null   int64  
 13  YearBuilt           1460 non-null   int64  
 14  YearRemodAdd        1460 non-null   int64  
 15  RoofStyle           1460 non-null   object 
 16  Exteri

In [19]:
pd.set_option('display.max_rows',100)
train_data.isna().sum()

MSSubClass              0
MSZoning                0
LotFrontage           259
LotArea                 0
LotShape                0
LandContour             0
LotConfig               0
Neighborhood            0
Condition1              0
BldgType                0
HouseStyle              0
OverallQual             0
OverallCond             0
YearBuilt               0
YearRemodAdd            0
RoofStyle               0
Exterior1st             0
Exterior2nd             0
MasVnrArea              8
ExterQual               0
ExterCond               0
Foundation              0
BsmtQual               37
BsmtCond               37
BsmtExposure           38
BsmtFinType1           37
BsmtFinSF1              0
BsmtFinType2           38
BsmtFinSF2              0
BsmtUnfSF               0
Heating                 0
HeatingQC               0
Electrical              1
1stFlrSF                0
2ndFlrSF                0
BedroomAbvGr            0
KitchenAbvGr            0
KitchenQual             0
TotRmsAbvGrd

In [ ]:
# # train_data = train_data.fillna(,axis = 1)
# # test_data = test_data.fillna(0,axis = 1)
# train_data = train_data.fillna(train_data.mean())
# test_data = test_data.fillna(test_data.mean())

# Filling null values Categorical

here 'c','n' are list contains categotical and numerical columns having null values in it

In [20]:
c = []
n = []
for i in train_data.columns:
    if train_data[i].isnull().sum() > 0:
        if (train_data[i].dtype) == 'O':
            c.append(i)
        else :
            n.append(i)
print(c,n)

['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'] ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']


here 'cc','nnn' are list contains categotical and numerical columns having null values in it

In [21]:
cc = []
nnn = []
for i in test_data.columns:
    if test_data[i].isnull().sum() > 0:
        if (test_data[i].dtype) == 'O':
            cc.append(i)
        else :
            nnn.append(i)
print(cc,nnn)

['MSZoning', 'Exterior1st', 'Exterior2nd', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'SaleType'] ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'n_bathrooms', 'area_with_basement']


filling nan with some values 


In [22]:
for i in c :
    if train_data[i].isnull().sum()> 0 and train_data[i][1] != 'NaN'  :
        train_data[i] = train_data[i].fillna(train_data[i][1])
    elif train_data[i].isnull().sum()> 0 and train_data[i][1] != 'NaN' :
        train_data[i] = train_data[i].fillna(train_data[i][1])

In [23]:
for i in cc :
    if test_data[i].isnull().sum()> 0 and test_data[i][1] != 'NaN'  :
        test_data[i] = test_data[i].fillna(test_data[i][1])
    elif test_data[i].isnull().sum()> 0 and test_data[i][10] != 'NaN' :
        test_data[i] = test_data[i].fillna(test_data[i][-1])

In [24]:
for i in cc :
    print('-*-*'*10)
    print(i)
    print(test_data[i].isnull().sum())
    print('___+++__'*8)

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
MSZoning
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Exterior1st
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Exterior2nd
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtQual
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtCond
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtExposure
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtFinType1
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtFinType2
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Kitch

In [25]:
for i in c :
    print('-*-*'*10)
    print(i)
    print(train_data[i].isnull().sum())
    print('___+++__'*8)

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtQual
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtCond
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtExposure
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtFinType1
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtFinType2
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Electrical
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
GarageType
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
GarageFinish
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Gar

# Filling null values continuous

In [26]:
for i in n :
    train_data[i] = train_data[i].fillna(np.mean(train_data[i]))
    #train_data[i] = train_data[i].fillna(np.mean(train_data[i]))

In [27]:
for i in nnn :
    test_data[i] = test_data[i].fillna(np.mean(test_data[i]))
    #train_data[i] = train_data[i].fillna(np.mean(train_data[i]))

In [28]:
for i in nnn :
    print('-*-*'*10)
    print(i)
    print(test_data[i].isnull().sum())
    print('___+++__'*8)

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotFrontage
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
MasVnrArea
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtFinSF1
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtFinSF2
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BsmtUnfSF
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
GarageYrBlt
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
GarageCars
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
GarageArea
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
n_bath

In [29]:
for i in n :
    print('-*-*'*10)
    print(i)
    print(train_data[i].isnull().sum())
    print('___+++__'*8)

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotFrontage
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
MasVnrArea
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
GarageYrBlt
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__


In [ ]:
# for i in test_data.columns :
#     print('-*-*'*10)
#     print(i)
#     print(test_data[i].isnull().sum())
#     print('___+++__'*8)

# fill nan. 

In [32]:
for i in n :
    train_data[i] = train_data[i].fillna(np.mean(train_data[i]))
    #train_data[i] = train_data[i].fillna(np.mean(train_data[i]))

In [31]:
for i in train_data.columns :
    print('-*-*'*10)
    print(i)
    print(train_data[i].isnull().sum())
    print('___+++__'*8)

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
MSSubClass
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
MSZoning
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotFrontage
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotArea
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotShape
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LandContour
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotConfig
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Neighborhood
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Condition1


In [33]:
for i in test_data.columns :
    print('-*-*'*10)
    print(i)
    print(test_data[i].isnull().sum())
    print('___+++__'*8)

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
MSSubClass
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
MSZoning
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotFrontage
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotArea
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotShape
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LandContour
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
LotConfig
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Neighborhood
0
___+++_____+++_____+++_____+++_____+++_____+++_____+++_____+++__
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Condition1


In [34]:
a = train_data[n]
a.corr()

,LotFrontage,MasVnrArea,GarageYrBlt
LotFrontage,1.000000,0.179283,0.064324
MasVnrArea,0.179283,1.000000,0.249367
GarageYrBlt,0.064324,0.249367,1.000000


In [ ]:
# c = []
# for i in train_data.columns:
#     if (train_data[i].dtype) == 'O':
#         c.append(i)
# print(c)
# print(len(c))

In [ ]:
# for i in c:
#     if train_data[i].isnull().sum() > 0:
#         train_data.fillna(np.mean(train_data[i]))
        

In [ ]:
# for i in train_data.columns :
#     print('-*-*'*10)
#     print(i)
#     print(train_data[i].isnull().sum())
#     print('___+++__'*8)

In [35]:
train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460.0,56.897260,42.300571,20.0,20.0,50.000000,70.00,190.0
LotFrontage,1460.0,70.049958,22.024023,21.0,60.0,70.049958,79.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.5,9478.500000,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.0,6.000000,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.0,5.000000,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.0,1973.000000,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.0,1994.000000,2004.00,2010.0
MasVnrArea,1460.0,103.685262,180.569112,0.0,0.0,0.000000,164.25,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.0,383.500000,712.25,5644.0
BsmtFinSF2,1460.0,46.549315,161.319273,0.0,0.0,0.000000,0.00,1474.0


In [36]:
train_data.describe(include = 'O').T

,count,unique,top,freq
MSZoning,1460,5,RL,1151
LotShape,1460,4,Reg,925
LandContour,1460,4,Lvl,1311
LotConfig,1460,5,Inside,1052
Neighborhood,1460,25,NAmes,225
Condition1,1460,9,Norm,1260
BldgType,1460,5,1Fam,1220
HouseStyle,1460,8,1Story,726
RoofStyle,1460,6,Gable,1141
Exterior1st,1460,15,VinylSd,515


In [37]:
test_data.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,n_bathrooms,area_with_basement
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,...,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,52.619342,...,1977.721217,1.766118,472.768861,93.174777,48.313914,24.243317,6.104181,2007.769705,2.226150,2532.568587
std,42.746880,20.561228,4955.517327,1.436812,1.113740,30.390071,21.130467,176.709824,455.111888,176.693301,...,25.714451,0.775679,216.974165,127.744882,68.883364,67.227765,2.722432,1.301740,0.830544,786.605431
min,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,0.000000,...,1895.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,1.000000,612.000000
25%,20.000000,60.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,0.000000,...,1960.500000,1.000000,318.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,1.500000,1998.000000
50%,50.000000,68.580357,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,351.000000,0.000000,...,1977.721217,2.000000,480.000000,0.000000,28.000000,0.000000,6.000000,2008.000000,2.000000,2431.000000
75%,70.000000,78.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,162.000000,752.000000,0.000000,...,2001.000000,2.000000,576.000000,168.000000,72.000000,0.000000,8.000000,2009.000000,3.000000,2982.000000
max,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,1526.000000,...,2207.000000,5.000000,1488.000000,1424.000000,742.000000,1012.000000,12.000000,2010.000000,7.000000,10190.000000


# training Label

In [38]:
train_out = train_data['SalePrice']
train_out = np.log(train_out)

In [39]:
train_out

0       12.247694
1       12.109011
2       12.317167
3       11.849398
4       12.429216
          ...    
1455    12.072541
1456    12.254863
1457    12.493130
1458    11.864462
1459    11.901583
Name: SalePrice, Length: 1460, dtype: float64

In [40]:
train_data.drop('SalePrice',axis = 1,inplace = True)
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,BldgType,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,SaleType,SaleCondition,n_bathrooms,area_with_basement
0,60,RL,65.0,8450,Reg,Lvl,Inside,CollgCr,Norm,1Fam,...,Y,0,61,0,2,2008,WD,Normal,3.5,2566
1,20,RL,80.0,9600,Reg,Lvl,FR2,Veenker,Feedr,1Fam,...,Y,298,0,0,5,2007,WD,Normal,2.5,2524
2,60,RL,68.0,11250,IR1,Lvl,Inside,CollgCr,Norm,1Fam,...,Y,0,42,0,9,2008,WD,Normal,3.5,2706
3,70,RL,60.0,9550,IR1,Lvl,Corner,Crawfor,Norm,1Fam,...,Y,0,35,272,2,2006,WD,Abnorml,2.0,2473
4,60,RL,84.0,14260,IR1,Lvl,FR2,NoRidge,Norm,1Fam,...,Y,192,84,0,12,2008,WD,Normal,3.5,3343


# testing label

In [41]:
test_actualout = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
test_actualout = test_actualout['SalePrice']
test_actualout = np.log(test_actualout)
test_actualout

0       12.039292
1       12.142911
2       12.120426
3       12.096913
4       11.923246
          ...    
1454    12.026235
1455    12.012420
1456    12.297842
1457    12.127702
1458    12.142823
Name: SalePrice, Length: 1459, dtype: float64

In [42]:
train_data

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,BldgType,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,SaleType,SaleCondition,n_bathrooms,area_with_basement
0,60,RL,65.0,8450,Reg,Lvl,Inside,CollgCr,Norm,1Fam,...,Y,0,61,0,2,2008,WD,Normal,3.5,2566
1,20,RL,80.0,9600,Reg,Lvl,FR2,Veenker,Feedr,1Fam,...,Y,298,0,0,5,2007,WD,Normal,2.5,2524
2,60,RL,68.0,11250,IR1,Lvl,Inside,CollgCr,Norm,1Fam,...,Y,0,42,0,9,2008,WD,Normal,3.5,2706
3,70,RL,60.0,9550,IR1,Lvl,Corner,Crawfor,Norm,1Fam,...,Y,0,35,272,2,2006,WD,Abnorml,2.0,2473
4,60,RL,84.0,14260,IR1,Lvl,FR2,NoRidge,Norm,1Fam,...,Y,192,84,0,12,2008,WD,Normal,3.5,3343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Reg,Lvl,Inside,Gilbert,Norm,1Fam,...,Y,0,40,0,8,2007,WD,Normal,2.5,2600
1456,20,RL,85.0,13175,Reg,Lvl,Inside,NWAmes,Norm,1Fam,...,Y,349,0,0,2,2010,WD,Normal,3.0,3615
1457,70,RL,66.0,9042,Reg,Lvl,Inside,Crawfor,Norm,1Fam,...,Y,0,60,0,5,2010,WD,Normal,2.0,3492
1458,20,RL,68.0,9717,Reg,Lvl,Inside,NAmes,Norm,1Fam,...,Y,366,0,112,4,2010,WD,Normal,2.0,2156


In [ ]:
# c = []
# for i in train_data.columns:
#     if (train_data[i].dtype) == 'O':
#         c.append(i)
# print(c)
# print(len(c))

In [43]:
# cate = [i for i in train_data.columns if train_data[i].dtype == object]   nextline code is same insted of doing all the stuff we simply ues select_dtypes
cate = train_data.select_dtypes(include  = 'O').columns
cate

Index(['MSZoning', 'LotShape', 'LandContour', 'LotConfig', 'Neighborhood',
       'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType',
       'SaleCondition'],
      dtype='object')

In [44]:
len(cate)

31

In [45]:
from sklearn.preprocessing import LabelEncoder
l = LabelEncoder()
for i in cate:
    train_data[i] = l.fit_transform(train_data[i])
    test_data[i] = l.fit_transform(test_data[i])

In [46]:
train_data

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,BldgType,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,SaleType,SaleCondition,n_bathrooms,area_with_basement
0,60,3,65.0,8450,3,3,4,5,2,0,...,2,0,61,0,2,2008,8,4,3.5,2566
1,20,3,80.0,9600,3,3,2,24,1,0,...,2,298,0,0,5,2007,8,4,2.5,2524
2,60,3,68.0,11250,0,3,4,5,2,0,...,2,0,42,0,9,2008,8,4,3.5,2706
3,70,3,60.0,9550,0,3,0,6,2,0,...,2,0,35,272,2,2006,8,0,2.0,2473
4,60,3,84.0,14260,0,3,2,15,2,0,...,2,192,84,0,12,2008,8,4,3.5,3343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,3,3,4,8,2,0,...,2,0,40,0,8,2007,8,4,2.5,2600
1456,20,3,85.0,13175,3,3,4,14,2,0,...,2,349,0,0,2,2010,8,4,3.0,3615
1457,70,3,66.0,9042,3,3,4,6,2,0,...,2,0,60,0,5,2010,8,4,2.0,3492
1458,20,3,68.0,9717,3,3,4,12,2,0,...,2,366,0,112,4,2010,8,4,2.0,2156


In [47]:
test_data

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,BldgType,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,SaleType,SaleCondition,n_bathrooms,area_with_basement
0,20,2,80.0,11622,3,3,4,12,1,0,...,2,140,0,0,6,2010,8,4,1.0,1778.0
1,20,3,81.0,14267,0,3,0,12,2,0,...,2,393,36,0,6,2010,8,4,1.5,2658.0
2,60,3,74.0,13830,0,3,4,8,2,0,...,2,212,34,0,3,2010,8,4,2.5,2557.0
3,60,3,78.0,9978,0,3,4,8,2,0,...,2,360,36,0,6,2010,8,4,2.5,2530.0
4,120,3,43.0,5005,0,1,4,22,2,4,...,2,0,82,0,1,2010,8,4,2.0,2560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,4,21.0,1936,3,3,4,10,2,3,...,2,0,0,0,6,2006,8,4,1.5,1638.0
1455,160,4,21.0,1894,3,3,4,10,2,4,...,2,0,24,0,4,2006,8,0,1.5,1638.0
1456,20,3,160.0,20000,3,3,4,11,2,0,...,2,474,0,0,9,2006,8,0,2.0,2448.0
1457,85,3,62.0,10441,3,3,4,11,2,0,...,2,80,32,0,7,2006,8,4,1.5,1882.0


In [48]:
type(train_data)

pandas.core.frame.DataFrame

In [49]:
train_data.corr()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,BldgType,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,SaleType,SaleCondition,n_bathrooms,area_with_basement
MSSubClass,1.000000,0.035900,-0.357056,-0.139781,0.119289,-0.002940,0.075910,-0.005985,-0.024762,0.746063,...,-0.059925,-0.012579,-0.006100,-0.012037,-0.013585,-0.021407,0.012464,-0.024940,0.151048,-0.079293
MSZoning,0.035900,1.000000,-0.106363,-0.034452,0.061887,-0.017854,-0.009895,-0.249679,-0.027874,0.005690,...,-0.100366,-0.009477,-0.154704,0.115509,-0.031496,-0.020628,0.097437,0.009494,-0.193097,-0.099498
LotFrontage,-0.357056,-0.106363,1.000000,0.306795,-0.144931,-0.075647,-0.181253,0.084545,-0.008483,-0.408564,...,0.089073,0.077106,0.137454,0.009790,0.010158,0.006768,-0.030846,0.058464,0.201473,0.428594
LotArea,-0.139781,-0.034452,0.306795,1.000000,-0.165315,-0.149083,-0.121161,0.044569,0.023846,-0.205721,...,0.015134,0.171698,0.084774,-0.018340,0.001205,-0.014261,0.012292,0.034169,0.204764,0.306814
LotShape,0.119289,0.061887,-0.144931,-0.165315,1.000000,0.085434,0.221102,-0.038894,-0.115003,0.116262,...,-0.113698,-0.166946,-0.075412,0.078213,-0.033455,0.036449,-0.000911,-0.038118,-0.201770,-0.205348
LandContour,-0.002940,-0.017854,-0.075647,-0.149083,0.085434,1.000000,-0.025527,0.019116,0.024801,0.051143,...,0.140921,-0.002879,0.040676,-0.058742,-0.011599,0.020507,-0.025754,0.033809,0.054199,-0.040956
LotConfig,0.075910,-0.009895,-0.181253,-0.121161,0.221102,-0.025527,1.000000,-0.036597,0.021457,0.107229,...,-0.045058,-0.035635,-0.054614,-0.070429,0.018902,-0.005992,0.014325,0.051579,-0.023609,-0.078250
Neighborhood,-0.005985,-0.249679,0.084545,0.044569,-0.038894,0.019116,-0.036597,1.000000,-0.025401,0.005993,...,0.040345,0.059370,0.144691,0.028803,0.023222,0.036697,-0.036956,0.021867,0.152690,0.173306
Condition1,-0.024762,-0.027874,-0.008483,0.023846,-0.115003,0.024801,0.021457,-0.025401,1.000000,-0.023501,...,0.105864,0.045338,0.085861,-0.079213,-0.009868,-0.009819,-0.002338,0.057747,0.095266,0.059446
BldgType,0.746063,0.005690,-0.408564,-0.205721,0.116262,0.051143,0.107229,0.005993,-0.023501,1.000000,...,0.059390,-0.013075,-0.037160,-0.114726,-0.025764,0.002006,-0.040306,-0.003530,0.080734,-0.107665


In [50]:
g = pd.DataFrame(train_data.dtypes,columns=['dtype'])
train_data

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,BldgType,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,SaleType,SaleCondition,n_bathrooms,area_with_basement
0,60,3,65.0,8450,3,3,4,5,2,0,...,2,0,61,0,2,2008,8,4,3.5,2566
1,20,3,80.0,9600,3,3,2,24,1,0,...,2,298,0,0,5,2007,8,4,2.5,2524
2,60,3,68.0,11250,0,3,4,5,2,0,...,2,0,42,0,9,2008,8,4,3.5,2706
3,70,3,60.0,9550,0,3,0,6,2,0,...,2,0,35,272,2,2006,8,0,2.0,2473
4,60,3,84.0,14260,0,3,2,15,2,0,...,2,192,84,0,12,2008,8,4,3.5,3343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,3,3,4,8,2,0,...,2,0,40,0,8,2007,8,4,2.5,2600
1456,20,3,85.0,13175,3,3,4,14,2,0,...,2,349,0,0,2,2010,8,4,3.0,3615
1457,70,3,66.0,9042,3,3,4,6,2,0,...,2,0,60,0,5,2010,8,4,2.0,3492
1458,20,3,68.0,9717,3,3,4,12,2,0,...,2,366,0,112,4,2010,8,4,2.0,2156


# Feature & Lable Scaling 

In [52]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train=scaler.fit_transform(train_data)
X_test = scaler.transform(test_data)

#  dataframe to torch dataloaders

In [ ]:
# use .values() sothat you get numpy array insted of  dataframe 
# this was the mistake 


In [53]:
# X_train = torch.tensor(train_data, dtype=torch.float32)
# y_train = torch.tensor(train_out, dtype=torch.float32)
# X_test = torch.tensor(test_data, dtype=torch.float32)
# y_test = torch.tensor(test_actualout, dtype=torch.float32)
from torch.autograd import Variable
X_train = Variable(torch.from_numpy(X_train)).float()
Y_train = torch.tensor(train_out).float()
X_test = Variable(torch.from_numpy(X_test)).float()
Y_test = torch.tensor(test_actualout).float()

In [ ]:
# #from torch.utils import Data
# #from torch.utils.data import Data 
# import tensorflow as tf
# tf.debugging.check_numerics(X_train, "Invalid tensor")

In [54]:
train_dataset = data.TensorDataset(X_train,Y_train)
batch = 64
train_dataloader = DataLoader(train_dataset,batch_size = batch,shuffle = True,)   

In [55]:
test_dataset = data.TensorDataset(X_test,Y_test)
batch_size = 64
test_dataloader = DataLoader(test_dataset,batch_size = batch_size,shuffle = True)

# Activation Function

In [56]:
import torch.nn as nn

relu = nn.ReLU()

In [57]:

(X_train)

tensor([[ 0.0734, -0.0455, -0.2294,  ...,  0.2085,  1.6423, -0.0084],
        [-0.8726, -0.0455,  0.4519,  ...,  0.2085,  0.3686, -0.0594],
        [ 0.0734, -0.0455, -0.0931,  ...,  0.2085,  1.6423,  0.1617],
        ...,
        [ 0.3099, -0.0455, -0.1840,  ...,  0.2085, -0.2683,  1.1163],
        [-0.8726, -0.0455, -0.0931,  ...,  0.2085, -0.2683, -0.5064],
        [-0.8726, -0.0455,  0.2248,  ...,  0.2085,  0.3686, -0.0740]])

# Neuralnetwork Architecture

In [58]:
class Neuralnetwork(nn.Module):
    def __init__(self,n_input,n_hidden,n_output):
        super(Neuralnetwork,self).__init__()
        #self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(n_input,n_hidden)
        self.relu = nn.ReLU()

        self.layer2 = nn.Linear(n_hidden,256)
        self.out = nn.Linear(256,n_output)
    def forward(self,x):
        #x = self.flatten(x)
        x = self.layer1(x)
        
        x = self.relu(x)
        x = self.layer2(x)
        x = relu(x)
        x = self.out(x)
        out_ =self.relu(x)
        return out_
    
    
   

# GPU & optimizer Setup 

In [64]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Neuralnetwork(X_train.shape[1],512,1)
model = model.to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
model

Neuralnetwork(
  (layer1): Linear(in_features=58, out_features=512, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=512, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=1, bias=True)
)

# Training 

In [65]:
def training(train_dataloader,model,loss_,optimizer):
    
    model.train()
    
    for batch,(X,y)in enumerate(train_dataloader):
        
        X = X.to(device)
        y = y.to(device)
        
        pred = model(X)
        #print('predicted value is ',pred)
        los = loss_(pred,y)
        #print('loss is as ',loss)
#3backpropagation
        los.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        
        if batch% 5 ==0:
            #print('batch number is *************       ' ,batch)
            print(f'loss:{los}')
#         with torch.no_grad():
#             pred = model(X_train.float())
#             train_mae = mean_absolute_error(y, pred.numpy())
#             train_rmse = np.sqrt(mean_squared_error(y, pred.numpy()))
#             print(train_rmse)

In [66]:
from sklearn.metrics import mean_absolute_error
#mean_absolute_error(y.detach().cpu().numpy(), pred.detach().cpu().numpy())
##.detach().cpu().numpy()   this is to move between cpu and gpu 

In [67]:
def test(model,data,loss_):
    model.eval()
    test_loss = 0
    size  = len(X_test)
    
    with torch.no_grad():
        
        for batch,(X,y) in enumerate(data):
            X = X.to(device)
            y = y.to(device)
    
            pred = model(X)
        
            batch_loss = loss_fn(pred, y)
            #print('test_loss is ',test_loss)
            #print('batch_loss is          ',batch_loss)
            #print('batch_loss of items is           ',   batch_loss.item())
            
            test_loss += batch_loss.item()
    
        average_loss = test_loss / size
        print(f'Test Loss: {average_loss:.4f}')

In [68]:
epoch = 20
i = 1
for t in range(epoch):
    print('-----------------epoch number -----',i)
    training(train_dataloader,model,loss_fn,optimizer)
    #print(y)
    test(model,test_dataloader,loss_fn)  
    i+=1

-----------------epoch number ----- 1
loss:144.04847717285156
loss:5.41775369644165
loss:29.934642791748047
loss:17.89410972595215
loss:26.69997787475586
Test Loss: 0.3838
-----------------epoch number ----- 2
loss:10.310723304748535
loss:21.02019691467285
loss:6.55150032043457
loss:9.743834495544434


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([52])) that is different to the input size (torch.Size([52, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([51])) that is different to the input size (torch.Size([51, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input,

loss:5.935700416564941
Test Loss: 0.0395
-----------------epoch number ----- 3
loss:5.933986186981201
loss:7.297842979431152
loss:9.55704402923584
loss:5.475194454193115
loss:3.0069074630737305
Test Loss: 0.2536
-----------------epoch number ----- 4
loss:4.9283294677734375
loss:3.836888551712036
loss:3.5848398208618164
loss:4.166872978210449
loss:2.7559633255004883
Test Loss: 0.0156
-----------------epoch number ----- 5
loss:4.085159778594971
loss:2.269608974456787
loss:3.751575231552124
loss:1.4343602657318115
loss:2.4441237449645996
Test Loss: 0.1362
-----------------epoch number ----- 6
loss:1.4002646207809448
loss:1.0214471817016602
loss:0.5492714047431946
loss:0.9612612724304199
loss:0.96379154920578
Test Loss: 0.0587
-----------------epoch number ----- 7
loss:0.6080375909805298
loss:0.8328609466552734
loss:0.6667919754981995
loss:0.6868924498558044
loss:0.6253740787506104
Test Loss: 0.0863
-----------------epoch number ----- 8
loss:0.6300904154777527
loss:0.5100692510604858
loss: